In [ ]:
# Safety shim for stray JSON cell execution
null = None
true = True
false = False


# ⚽ Soccer Player Highlight Reel Generator - Production Grade

This notebook implements a complete, production-grade, end-to-end pipeline for generating personalized soccer player highlight reels. This is an advanced version with sophisticated algorithms for each stage.

## 🚀 Advanced Features
- Player Detection: High-performance YOLOv8 for person detection.
- Multi-Object Tracking: Full implementation of ByteTrack with a Kalman Filter for motion prediction and Hungarian Algorithm for association.
- Long-term Re-Identification: A hybrid system using a Deep CNN for appearance embeddings combined with color histograms and Jersey OCR.
- Intelligent Event Detection: Advanced kinematics-based event detector.
- Professional Video Assembly: Integration of PySceneDetect to find natural scene boundaries for clean clip extraction, stitched with FFmpeg.

## ⚡ How to Use
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Run All Cells: Runtime → Run all
3. Upload Video when prompted
4. Wait for processing
5. Download results automatically


In [1]:
print("Installing dependencies...")
!pip install ultralytics torch torchvision opencv-python-headless easyocr scikit-learn numpy pandas tqdm pillow 'scenedetect[opencv]' --quiet
import torch, os
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected - using CPU (will be much slower)")
os.makedirs('/content/videos', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)
os.makedirs('/content/temp_clips', exist_ok=True)
print("Setup complete!")


Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files
import shutil
print("Please upload your soccer match video file.")
uploaded = files.upload()
video_path = None
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov')):
        destination_path = f'/content/videos/{filename}'
        shutil.move(filename, destination_path)
        print(f"Video uploaded: {destination_path}")
        video_path = destination_path
        break
if not video_path:
    print("No video file found. Please upload an MP4, AVI, or MOV file.")


Please upload your soccer match video file.


Saving 9.mp4 to 9.mp4
Video uploaded: /content/videos/9.mp4


In [3]:
import cv2
import torch
import numpy as np
import json
from ultralytics import YOLO
import easyocr
from tqdm.notebook import tqdm
import math
from sklearn.metrics.pairwise import cosine_similarity
import subprocess
from scipy.optimize import linear_sum_assignment
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Tuple
from scenedetect import open_video, SceneManager
from scenedetect.detectors import ContentDetector
print("Modules imported.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Modules imported.


In [4]:
class SoccerPlayerDetector:
    def __init__(self, model_name: str = 'yolov8n.pt', conf_thresh: float = 0.35):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_name)
        self.model.to(self.device)
        self.conf_thresh = conf_thresh
    def process_video(self, video_path: str, output_path: str) -> List[Dict]:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        all_detections = []
        with tqdm(total=total_frames, desc="Stage 1: Detecting players") as pbar:
            for frame_idx in range(total_frames):
                ret, frame = cap.read()
                if not ret:
                    break
                results = self.model(frame, classes=[0], imgsz=960, verbose=False)
                detections = []
                if len(results) > 0 and results[0].boxes is not None:
                    for box in results[0].boxes:
                        if box.conf[0] >= self.conf_thresh:
                            detections.append({'bbox': [int(coord) for coord in box.xyxy[0].tolist()], 'confidence': float(box.conf[0])})
                all_detections.append({"frame_id": frame_idx, "detections": detections})
                pbar.update(1)
        cap.release()
        with open(output_path, 'w') as f:
            json.dump(all_detections, f, indent=2)
        return all_detections


SyntaxError: invalid syntax (ipython-input-2274920068.py, line 1)

In [ ]:
class KalmanFilter:
    def __init__(self):
        self.kf = cv2.KalmanFilter(7, 4)
        self.kf.measurementMatrix = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]], np.float32)
        self.kf.transitionMatrix = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,1], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]], np.float32)
        cv2.setIdentity(self.kf.processNoiseCov, 1e-2)
        cv2.setIdentity(self.kf.measurementNoiseCov, 1e-1)
        cv2.setIdentity(self.kf.errorCovPost, 1)
    def predict(self):
        return self.kf.predict()
    def update(self, bbox: List[float]):
        x, y, w, h = bbox
        measurement = np.array([x + w / 2, y + h / 2, w / h, h], dtype=np.float32).reshape(4, 1)
        self.kf.correct(measurement)
    def init(self, bbox: List[float]):
        x, y, w, h = bbox
        self.kf.statePost = np.array([x + w / 2, y + h / 2, w / h, h, 0, 0, 0], dtype=np.float32).reshape(7, 1)
class STrack:
    def __init__(self, tlwh: List[float], score: float):
        self.tlwh = np.asarray(tlwh, dtype=float)
        self.score = score
        self.kalman_filter = KalmanFilter()
        self.kalman_filter.init(self.tlwh)
        self.track_id = 0
        self.state = 'new'
        self.is_activated = False
        self.frame_id = 0
        self.start_frame = 0
        self.time_since_update = 0
    def activate(self, frame_id: int, track_id: int):
        self.track_id = track_id
        self.frame_id = frame_id
        self.start_frame = frame_id
        self.state = 'tracked'
        self.is_activated = True
    def re_activate(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0
    def predict(self):
        if self.state != 'tracked':
            self.kalman_filter.kf.statePost[6,0] = 0
        self.kalman_filter.predict()
    def update(self, new_track, frame_id: int):
        self.tlwh = new_track.tlwh
        self.score = new_track.score
        self.kalman_filter.update(self.tlwh)
        self.state = 'tracked'
        self.is_activated = True
        self.frame_id = frame_id
        self.time_since_update = 0
    @property
    def tlbr(self) -> List[float]:
        x, y, w, h = self.tlwh
        return [x, y, x + w, y + h]
def iou_distance(atracks: List[STrack], btracks: List[STrack]) -> np.ndarray:
    atlbrs = np.array([track.tlbr for track in atracks]) if atracks else np.empty((0, 4))
    btlbrs = np.array([track.tlbr for track in btracks]) if btracks else np.empty((0, 4))
    ious = np.zeros((len(atlbrs), len(btlbrs)), dtype=float)
    if len(atlbrs) == 0 or len(btlbrs) == 0:
        return 1 - ious
    for i, a in enumerate(atlbrs):
        for j, b in enumerate(btlbrs):
            box_inter = [max(a[0], b[0]), max(a[1], b[1]), min(a[2], b[2]), min(a[3], b[3])]
            inter_area = max(0, box_inter[2] - box_inter[0]) * max(0, box_inter[3] - box_inter[1])
            union_area = (a[2] - a[0]) * (a[3] - a[1]) + (b[2] - b[0]) * (b[3] - b[1]) - inter_area
            if union_area > 0:
                ious[i, j] = inter_area / union_area
    return 1 - ious
class ByteTrack:
    def __init__(self, high_thresh: float = 0.6, low_thresh: float = 0.1, max_time_lost: int = 30):
        self.tracked_stracks: List[STrack] = []
        self.lost_stracks: List[STrack] = []
        self.removed_stracks: List[STrack] = []
        self.frame_id = 0
        self.track_id_count = 0
        self.high_thresh = high_thresh
        self.low_thresh = low_thresh
        self.max_time_lost = max_time_lost
    def update(self, detections: List[Dict]) -> List[Dict]:
        self.frame_id += 1
        activated_starcks, refind_stracks, lost_stracks, removed_stracks = [], [], [], []
        dets_high = [d for d in detections if d['confidence'] >= self.high_thresh]
        dets_low = [d for d in detections if self.low_thresh <= d['confidence'] < self.high_thresh]
        stracks_high = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_high]
        stracks_low = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low]
        for strack in self.tracked_stracks: strack.predict()
        dists = iou_distance(self.tracked_stracks, stracks_high)
        matches, u_track, u_detection = self.linear_assignment(dists, 0.8)
        for i, j in matches:
            track = self.tracked_stracks[i]
            det = stracks_high[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)
        unmatched_tracks = [self.tracked_stracks[i] for i in u_track]
        dists = iou_distance(unmatched_tracks, stracks_low)
        matches, u_track, u_detection_low = self.linear_assignment(dists, 0.5)
        for i, j in matches:
            track = unmatched_tracks[i]
            det = stracks_low[j]
            track.update(det, self.frame_id)
            activated_starcks.append(track)
        for i in u_track:
            track = unmatched_tracks[i]
            track.state = 'lost'
            lost_stracks.append(track)
        for i in u_detection:
            track = stracks_high[i]
            if track.score >= self.high_thresh:
                self.track_id_count += 1
                track.activate(self.frame_id, self.track_id_count)
                activated_starcks.append(track)
        self.tracked_stracks = [t for t in self.tracked_stracks if t.state == 'tracked'] + activated_starcks
        self.lost_stracks = [t for t in self.lost_stracks if t.time_since_update <= self.max_time_lost] + lost_stracks
        output = [{'track_id': t.track_id, 'bbox': [int(x) for x in t.tlbr]} for t in self.tracked_stracks if t.is_activated]
        return output
    def linear_assignment(self, cost_matrix, thresh):
        import numpy as np
        if getattr(cost_matrix, 'size', 0) == 0:
            rows = cost_matrix.shape[0] if hasattr(cost_matrix, 'shape') else 0
            cols = cost_matrix.shape[1] if hasattr(cost_matrix, 'shape') else 0
            return [], list(range(rows)), list(range(cols))
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matches = [(r, c) for r, c in zip(row_ind, col_ind) if cost_matrix[r, c] < thresh]
        u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]
        u_detection = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]
        return matches, u_track, u_detection


In [ ]:
class PlayerFeatureExtractor(nn.Module):
    def __init__(self, embedding_dim=128):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, embedding_dim)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.normalize(x, p=2, dim=1)
class PlayerReID:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.feature_extractor = PlayerFeatureExtractor().to(self.device).eval()
        self.ocr = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.global_players = {}
        self.next_permanent_id = 1
        self.similarity_threshold = 0.45
        self.jersey_bonus = 0.4
    def get_features(self, patch):
        hsv = cv2.cvtColor(cv2.resize(patch, (64, 64)), cv2.COLOR_BGR2HSV)
        color_hist = cv2.normalize(cv2.calcHist([hsv], [0, 1], None, [30, 32], [0, 180, 0, 256]), None).flatten()
        img_tensor = torch.from_numpy(cv2.resize(patch, (64, 64))).permute(2, 0, 1).float().div(255).unsqueeze(0).to(self.device)
        with torch.no_grad():
            deep_features = self.feature_extractor(img_tensor).cpu().numpy().flatten()
        jersey = None
        try:
            gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
            results = self.ocr.readtext(gray, allowlist='0123456789', detail=0, paragraph=False)
            if results and results[0].isdigit() and 1 <= len(results[0]) <= 2:
                jersey = int(results[0])
        except Exception:
            pass
        return {'color': color_hist, 'deep': deep_features, 'jersey': jersey}
    def process_tracklets(self, tracklets_path, video_path, output_path):
        with open(tracklets_path, 'r') as f:
            all_tracklets = json.load(f)
        cap = cv2.VideoCapture(video_path)
        frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        long_tracks = []
        current_frame_index = -1
        for frame_data in tqdm(all_tracklets, desc='Stage 3: Re-identifying players'):
            target_index = frame_data['frame_id']
            while current_frame_index < target_index:
                ret, frame = cap.read()
                if not ret:
                    break
                current_frame_index += 1
            if current_frame_index != target_index:
                break
            frame_tracks = {'frame_id': target_index, 'players': []}
            for track in frame_data['tracks']:
                x1, y1, x2, y2 = track['bbox']
                x1 = max(0, min(frame_w - 1, int(x1)))
                y1 = max(0, min(frame_h - 1, int(y1)))
                x2 = max(0, min(frame_w, int(x2)))
                y2 = max(0, min(frame_h, int(y2)))
                if x2 <= x1 or y2 <= y1:
                    continue
                patch = frame[y1:y2, x1:x2]
                if patch.size == 0:
                    continue
                current_features = self.get_features(patch)
                best_id = None
                best_score = self.similarity_threshold
                for pid, p_info in self.global_players.items():
                    color_sim = cv2.compareHist(current_features['color'], p_info['features']['color'], cv2.HISTCMP_CORREL)
                    deep_sim = float(cosine_similarity(current_features['deep'].reshape(1, -1), p_info['features']['deep'].reshape(1, -1))[0][0])
                    sim = 0.4 * color_sim + 0.6 * deep_sim
                    if current_features['jersey'] is not None and current_features['jersey'] == p_info['features']['jersey']:
                        sim += self.jersey_bonus
                    if sim > best_score:
                        best_score = sim
                        best_id = pid
                if best_id is None:
                    best_id = self.next_permanent_id
                    self.next_permanent_id += 1
                if best_id in self.global_players:
                    alpha = 0.12
                    self.global_players[best_id]['features']['color'] = (1 - alpha) * self.global_players[best_id]['features']['color'] + alpha * current_features['color']
                    self.global_players[best_id]['features']['deep'] = (1 - alpha) * self.global_players[best_id]['features']['deep'] + alpha * current_features['deep']
                    if self.global_players[best_id]['features']['jersey'] is None and current_features['jersey'] is not None:
                        self.global_players[best_id]['features']['jersey'] = current_features['jersey']
                else:
                    self.global_players[best_id] = {'features': current_features}
                frame_tracks['players'].append({'permanent_id': best_id, 'bbox': [x1, y1, x2, y2], 'jersey': current_features['jersey']})
            long_tracks.append(frame_tracks)
        cap.release()
        with open(output_path, 'w') as f:
            json.dump(long_tracks, f, indent=2)
        return long_tracks


In [ ]:
{
 "nbformat": 4,
 "nbformat_minor": 5,
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "cells": [
  {
   "cell_type": "markdown",
   "source": [
    "# ⚽ Soccer Player Highlight Reel Generator - Production Grade\n",
    "\n",
    "This notebook implements a complete, production-grade, end-to-end pipeline for generating personalized soccer player highlight reels. This is an advanced version with sophisticated algorithms for each stage.\n",
    "\n",
    "## 🚀 Advanced Features\n",
    "- **Player & Ball Detection**: High-performance YOLOv8 for robust object detection.\n",
    "- **Multi-Object Tracking**: Full implementation of **ByteTrack** with a **Kalman Filter** for motion prediction and Hungarian Algorithm for association.\n",
    "- **Long-term Re-Identification**: A hybrid system using a **Deep CNN** for appearance embeddings combined with color histograms and validated Jersey OCR.\n",
    "- **Team Classification**: Automatic K-Means clustering on jersey colors to assign players to teams.\n",
    "- **Intelligent Event Detection**: A sophisticated model that analyzes player/ball kinematics, interactions, and clustering to detect and score events like shots, goals, passes, and tackles.\n",
    "- **Professional Video Assembly**: Integration of **PySceneDetect** to find natural scene boundaries, with dynamic title overlays for events.\n",
    "\n",
    "## ⚡ How to Use\n",
    "1. **Enable GPU**: Go to `Runtime` → `Change runtime type` and select `T4 GPU`.\n",
    "2. **Adjust Configuration**: Modify the parameters in the configuration cell below (e.g., `TARGET_JERSEY_NUMBER`).\n",
    "3. **Run All Cells**: Click `Runtime` → `Run all`.\n",
    "4. **Upload Video**: An upload prompt will appear. Select your soccer match video.\n",
    "5. **Wait**: The pipeline will process the video. This is a computationally intensive process.\n",
    "6. **Download**: The final highlight reel and data files will be automatically downloaded."
   ],
   "metadata": {
    "id": "header"
   }
  },
  {
   "cell_type": "markdown",
   "source": [
    "## ⚙️ 1. Configuration"
   ],
   "metadata": {
    "id": "config_markdown"
   }
  },
  {
   "cell_type": "code",
   "source": [
    "# --- Main Configuration ---\n",
    "TARGET_JERSEY_NUMBER = 1  # Jersey number of the player you want to highlight.\n",
    "TOP_N_EVENTS = 10         # The number of top-scoring events to include in the final reel.\n",
    "YOLO_MODEL_SIZE = 'yolov8m.pt' # 'yolov8n.pt' (faster) or 'yolov8m.pt' (more accurate)\n",
    "\n",
    "# --- Fine-Tuning Parameters (Advanced) ---\n",
    "DETECTION_CONF_THRESHOLD = 0.4\n",
    "MIN_BBOX_AREA = 1000 # Minimum pixel area for a detection to be considered valid.\n",
    "\n",
    "TRACKING_HIGH_THRESH = 0.5\n",
    "TRACKING_LOW_THRESH = 0.1\n",
    "TRACKING_MAX_TIME_LOST = 60 # Frames a track can be lost before being discarded.\n",
    "\n",
    "REID_SIMILARITY_THRESHOLD = 0.4 # Lower is more lenient for re-identification.\n",
    "REID_JERSEY_BONUS = 0.5\n",
    "\n",
    "EVENT_SCORE_WEIGHTS = {\n",
    "    'goal': 1.0,\n",
    "    'shot': 0.8,\n",
    "    'tackle': 0.6,\n",
    "    'pass': 0.3,\n",
    "    'dribble': 0.4,\n",
    "    'sprint': 0.2\n",
    "}\n",
    "\n",
    "print(\"✅ Configuration set.\")"
   ],
   "metadata": {
    "id": "config_cell"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "source": [
    "## 🔧 2. Setup & Installation"
   ],
   "metadata": {
    "id": "setup_markdown"
   }
  },
  {
   "cell_type": "code",
   "source": [
    "print(\"Installing dependencies...\")\n",
    "!pip install ultralytics torch torchvision opencv-python-headless easyocr scikit-learn numpy pandas tqdm pillow 'scenedetect[opencv]' --quiet\n",
    "\n",
    "import torch\n",
    "import os\n",
    "\n",
    "print(f\"CUDA available: {torch.cuda.is_available()}\")\n",
    "if torch.cuda.is_available():\n",
    "    print(f\"GPU: {torch.cuda.get_device_name(0)}\")\n",
    "else:\n",
    "    print(\"⚠️ No GPU detected - using CPU (will be much slower)\")\n",
    "\n",
    "os.makedirs('/content/videos', exist_ok=True)\n",
    "os.makedirs('/content/output', exist_ok=True)\n",
    "os.makedirs('/content/temp_clips', exist_ok=True)\n",
    "\n",
    "print(\"✅ Setup complete!\")"
   ],
   "metadata": {
    "id": "install_deps"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "source": [
    "## 📁 3. Upload Your Video"
   ],
   "metadata": {
    "id": "upload_markdown"
   }
  },
  {
   "cell_type": "code",
   "source": [
    "from google.colab import files\n",
    "import shutil\n",
    "\n",
    "print(\"Please upload your soccer match video file.\")\n",
    "uploaded = files.upload()\n",
    "\n",
    "video_path = None\n",
    "for filename in uploaded.keys():\n",
    "    if filename.lower().endswith(('.mp4', '.avi', '.mov')):\n",
    "        destination_path = f'/content/videos/{filename}'\n",
    "        shutil.move(filename, destination_path)\n",
    "        print(f\"✅ Video uploaded: {destination_path}\")\n",
    "        video_path = destination_path\n",
    "        break\n",
    "\n",
    "if not video_path:\n",
    "    print(\"❌ No video file found. Please upload an MP4, AVI, or MOV file.\")"
   ],
   "metadata": {
    "id": "upload_video"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "source": [
    "## 🚀 4. Run the Full End-to-End Pipeline"
   ],
   "metadata": {
    "id": "pipeline_header"
   }
  },
  {
   "cell_type": "code",
   "source": [
    "import cv2\n",
    "import torch\n",
    "import numpy as np\n",
    "import json\n",
    "from ultralytics import YOLO\n",
    "import easyocr\n",
    "from tqdm.notebook import tqdm\n",
    "import math\n",
    "from sklearn.metrics.pairwise import cosine_similarity\n",
    "from sklearn.cluster import KMeans\n",
    "import subprocess\n",
    "from scipy.optimize import linear_sum_assignment\n",
    "import torch.nn as nn\n",
    "import torch.nn.functional as F\n",
    "from typing import List, Dict, Tuple\n",
    "from scenedetect import open_video, SceneManager\n",
    "from scenedetect.detectors import ContentDetector\n",
    "from collections import Counter\n",
    "\n",
    "print(\"✅ All modules imported successfully!\")"
   ],
   "metadata": {
    "id": "import_modules"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "class SoccerObjectDetector:\n",
    "    \"\"\"Detects players and the ball in a video using YOLOv8.\"\"\"\n",
    "    def __init__(self, model_name: str = 'yolov8m.pt', conf_thresh: float = 0.4, min_area: int = 1000):\n",
    "        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "        self.model = YOLO(model_name)\n",
    "        self.model.to(self.device)\n",
    "        self.conf_thresh = conf_thresh\n",
    "        self.min_area = min_area\n",
    "        # YOLO classes: 0 is person, 32 is sports ball\n",
    "        self.target_classes = [0, 32]\n",
    "        print(f\"Detector initialized on {self.device} with model {model_name}\")\n",
    "\n",
    "    def process_video(self, video_path: str, output_path: str) -> List[Dict]:\n",
    "        cap = cv2.VideoCapture(video_path)\n",
    "        if not cap.isOpened():\n",
    "            print(f\"Error: Could not open video {video_path}\")\n",
    "            return []\n",
    "        \n",
    "        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))\n",
    "        all_detections = []\n",
    "        \n",
    "        with tqdm(total=total_frames, desc=\"Stage 1: Detecting Players & Ball\") as pbar:\n",
    "            for frame_idx in range(total_frames):\n",
    "                ret, frame = cap.read()\n",
    "                if not ret: break\n",
    "                \n",
    "                results = self.model(frame, classes=self.target_classes, imgsz=960, verbose=False)\n",
    "                \n",
    "                frame_detections = {'players': [], 'ball': None}\n",
    "                if len(results) > 0 and results[0].boxes is not None:\n",
    "                    for box in results[0].boxes:\n",
    "                        if box.conf[0] >= self.conf_thresh:\n",
    "                            x1, y1, x2, y2 = [int(c) for c in box.xyxy[0].tolist()]\n",
    "                            if (x2 - x1) * (y2 - y1) < self.min_area: continue\n",
    "                            \n",
    "                            detection = {'bbox': [x1, y1, x2, y2], 'confidence': float(box.conf[0])}\n",
    "                            if int(box.cls[0]) == 0: # Person\n",
    "                                frame_detections['players'].append(detection)\n",
    "                            elif int(box.cls[0]) == 32: # Sports Ball\n",
    "                                if frame_detections['ball'] is None or detection['confidence'] > frame_detections['ball']['confidence']:\n",
    "                                    frame_detections['ball'] = detection\n",
    "                \n",
    "                all_detections.append({\"frame_id\": frame_idx, \"detections\": frame_detections})\n",
    "                pbar.update(1)\n",
    "        \n",
    "        cap.release()\n",
    "        with open(output_path, 'w') as f: json.dump(all_detections, f, indent=2)\n",
    "        return all_detections"
   ],
   "metadata": {
    "id": "detector_class"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "# Note: The Kalman Filter and STrack classes remain largely the same as they are standard implementations.\n",
    "class KalmanFilter:\n",
    "    def __init__(self):\n",
    "        self.kf = cv2.KalmanFilter(7, 4)\n",
    "        self.kf.measurementMatrix = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]], np.float32)\n",
    "        self.kf.transitionMatrix = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,1], [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]], np.float32)\n",
    "        cv2.setIdentity(self.kf.processNoiseCov, 1e-2)\n",
    "        cv2.setIdentity(self.kf.measurementNoiseCov, 1e-1)\n",
    "        cv2.setIdentity(self.kf.errorCovPost, 1)\n",
    "    def predict(self): return self.kf.predict()\n",
    "    def update(self, bbox): self.kf.correct(np.array([bbox[0]+bbox[2]/2, bbox[1]+bbox[3]/2, bbox[2]/bbox[3], bbox[3]], dtype=np.float32).reshape(4,1))\n",
    "    def init(self, bbox): self.kf.statePost = np.array([bbox[0]+bbox[2]/2, bbox[1]+bbox[3]/2, bbox[2]/bbox[3], bbox[3], 0, 0, 0], dtype=np.float32).reshape(7,1)\n",
    "\n",
    "class STrack:\n",
    "    def __init__(self, tlwh, score): self.tlwh, self.score, self.kalman_filter, self.track_id, self.state, self.is_activated, self.frame_id, self.start_frame, self.time_since_update = np.asarray(tlwh, dtype=float), score, KalmanFilter(), 0, 'new', False, 0, 0, 0; self.kalman_filter.init(self.tlwh)\n",
    "    def activate(self, frame_id, track_id): self.track_id, self.frame_id, self.start_frame, self.state, self.is_activated = track_id, frame_id, frame_id, 'tracked', True\n",
    "    def re_activate(self, new_track, frame_id): self.tlwh, self.score, self.kalman_filter.update(self.tlwh), self.state, self.is_activated, self.frame_id, self.time_since_update = new_track.tlwh, new_track.score, 'tracked', True, frame_id, 0\n",
    "    def predict(self): self.kalman_filter.predict()\n",
    "    def update(self, new_track, frame_id): self.tlwh, self.score, self.kalman_filter.update(self.tlwh), self.state, self.is_activated, self.frame_id, self.time_since_update = new_track.tlwh, new_track.score, 'tracked', True, frame_id, 0\n",
    "    @property\n",
    "    def tlbr(self): x, y, w, h = self.tlwh; return [x, y, x + w, y + h]\n",
    "\n",
    "def iou_distance(atracks, btracks):\n",
    "    if not atracks or not btracks: return np.empty((0, 0))\n",
    "    atlbrs, btlbrs = np.array([t.tlbr for t in atracks]), np.array([t.tlbr for t in btracks])\n",
    "    ious = np.zeros((len(atlbrs), len(btlbrs)))\n",
    "    for i, a in enumerate(atlbrs):\n",
    "        for j, b in enumerate(btlbrs):\n",
    "            i_area = max(0, min(a[2],b[2]) - max(a[0],b[0])) * max(0, min(a[3],b[3]) - max(a[1],b[1]))\n",
    "            u_area = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - i_area\n",
    "            if u_area > 0: ious[i,j] = i_area / u_area\n",
    "    return 1 - ious\n",
    "\n",
    "class ByteTrack:\n",
    "    def __init__(self, high_thresh=0.6, low_thresh=0.1, max_time_lost=30):\n",
    "        self.tracked, self.lost, self.removed = [], [], []\n",
    "        self.frame_id, self.track_id_count = 0, 0\n",
    "        self.high, self.low, self.max_lost = high_thresh, low_thresh, max_time_lost\n",
    "\n",
    "    def update(self, detections):\n",
    "        self.frame_id += 1\n",
    "        activated, refind, lost, removed = [], [], [], []\n",
    "        dets_high = [d for d in detections if d['confidence'] >= self.high]\n",
    "        dets_low = [d for d in detections if self.low < d['confidence'] < self.high]\n",
    "        stracks_high = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_high]\n",
    "        stracks_low = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low]\n",
    "        \n",
    "        for t in self.tracked: t.predict()\n",
    "        dists = iou_distance(self.tracked, stracks_high)\n",
    "        matches, u_track, u_det = self.linear_assignment(dists, 0.8)\n",
    "        for i, j in matches: self.tracked[i].update(stracks_high[j], self.frame_id); activated.append(self.tracked[i])\n",
    "        \n",
    "        unmatched = [self.tracked[i] for i in u_track]\n",
    "        dists = iou_distance(unmatched, stracks_low)\n",
    "        matches, u_track, _ = self.linear_assignment(dists, 0.5)\n",
    "        for i, j in matches: unmatched[i].update(stracks_low[j], self.frame_id); activated.append(unmatched[i])\n",
    "        \n",
    "        for i in u_track: unmatched[i].state = 'lost'; lost.append(unmatched[i])\n",
    "        for i in u_det: \n",
    "            if stracks_high[i].score >= self.high: self.track_id_count += 1; stracks_high[i].activate(self.frame_id, self.track_id_count); activated.append(stracks_high[i])\n",
    "        \n",
    "        self.tracked = [t for t in self.tracked if t.state == 'tracked'] + activated\n",
    "        self.lost = [t for t in self.lost if t.time_since_update <= self.max_lost] + lost\n",
    "        return [{'track_id': t.track_id, 'bbox': [int(x) for x in t.tlbr]} for t in self.tracked if t.is_activated]\n",
    "\n",
    "    def linear_assignment(self, cost_matrix, thresh):\n",
    "        if cost_matrix.size == 0: return [], list(range(cost_matrix.shape[0])), list(range(cost_matrix.shape[1]))\n",
    "        rows, cols = linear_sum_assignment(cost_matrix)\n",
    "        matches = [(r,c) for r,c in zip(rows,cols) if cost_matrix[r,c] < thresh]\n",
    "        u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]\n",
    "        u_det = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]\n",
    "        return matches, u_track, u_det"
   ],
   "metadata": {
    "id": "bytetrack_class"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "class PlayerFeatureExtractor(nn.Module):\n",
    "    def __init__(self, embedding_dim=128):\n",
    "        super().__init__()\n",
    "        # Increased complexity for better feature extraction\n",
    "        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)\n",
    "        self.bn1 = nn.BatchNorm2d(32)\n",
    "        self.pool = nn.MaxPool2d(2, 2)\n",
    "        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)\n",
    "        self.bn2 = nn.BatchNorm2d(64)\n",
    "        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)\n",
    "        self.bn3 = nn.BatchNorm2d(128)\n",
    "        self.fc1 = nn.Linear(128 * 8 * 8, 512)\n",
    "        self.fc2 = nn.Linear(512, embedding_dim)\n",
    "    \n",
    "    def forward(self, x):\n",
    "        x = self.pool(F.relu(self.bn1(self.conv1(x))))\n",
    "        x = self.pool(F.relu(self.bn2(self.conv2(x))))\n",
    "        x = self.pool(F.relu(self.bn3(self.conv3(x))))\n",
    "        x = x.view(-1, 128 * 8 * 8)\n",
    "        x = F.relu(self.fc1(x))\n",
    "        x = self.fc2(x)\n",
    "        return F.normalize(x, p=2, dim=1)\n",
    "\n",
    "class PlayerReID:\n",
    "    def __init__(self):\n",
    "        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'\n",
    "        self.feature_extractor = PlayerFeatureExtractor().to(self.device).eval()\n",
    "        self.ocr = easyocr.Reader(['en'], gpu=torch.cuda.is_available())\n",
    "        self.global_players = {}\n",
    "        self.next_permanent_id = 1\n",
    "        self.similarity_threshold = REID_SIMILARITY_THRESHOLD\n",
    "        self.jersey_bonus = REID_JERSEY_BONUS\n",
    "        self.ocr_validation_buffer = 3 # Number of consistent reads to confirm a jersey number\n",
    "\n",
    "    def get_features(self, patch):\n",
    "        hsv = cv2.cvtColor(cv2.resize(patch, (64, 64)), cv2.COLOR_BGR2HSV)\n",
    "        color_hist = cv2.normalize(cv2.calcHist([hsv], [0, 1], None, [16, 16], [0, 180, 0, 256]), None).flatten()\n",
    "        img_tensor = torch.from_numpy(cv2.resize(patch, (64, 64))).permute(2, 0, 1).float().div(255).unsqueeze(0).to(self.device)\n",
    "        with torch.no_grad(): deep_features = self.feature_extractor(img_tensor).cpu().numpy().flatten()\n",
    "        jersey = None\n",
    "        try:\n",
    "            gray = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)\n",
    "            results = self.ocr.readtext(gray, allowlist='0123456789', detail=0, paragraph=False)\n",
    "            if results and results[0].isdigit() and 1 <= len(results[0]) <= 2: jersey = int(results[0])\n",
    "        except: pass\n",
    "        return {'color': color_hist, 'deep': deep_features, 'jersey': jersey, 'dominant_color': self.get_dominant_color(patch)}\n",
    "\n",
    "    def get_dominant_color(self, patch):\n",
    "        pixels = cv2.resize(patch, (10, 10)).reshape(-1, 3)\n",
    "        kmeans = KMeans(n_clusters=2, n_init='auto')\n",
    "        kmeans.fit(pixels)\n",
    "        # Exclude white/black/gray colors from being the dominant color\n",
    "        colors = [c for c in kmeans.cluster_centers_ if np.std(c) > 15]\n",
    "        return colors[0] if colors else kmeans.cluster_centers_[0]\n",
    "\n",
    "    def process_tracklets(self, tracklets_path, video_path, output_path):\n",
    "        with open(tracklets_path, 'r') as f: all_tracklets = json.load(f)\n",
    "        cap = cv2.VideoCapture(video_path)\n",
    "        long_tracks = []\n",
    "        \n",
    "        all_player_colors = []\n",
    "\n",
    "        for frame_data in tqdm(all_tracklets, desc=\"Stage 3: Re-identifying players\"):\n",
    "            frame_id = frame_data['frame_id']\n",
    "            ret, frame = cap.read()\n",
    "            if not ret: break\n",
    "            \n",
    "            frame_tracks = {\"frame_id\": frame_id, \"players\": []}\n",
    "            for track in frame_data['tracks']:\n",
    "                x1, y1, x2, y2 = track['bbox']\n",
    "                patch = frame[y1:y2, x1:x2]\n",
    "                if patch.size == 0: continue\n",
    "                \n",
    "                current_features = self.get_features(patch)\n",
    "                best_id, best_score = None, self.similarity_threshold\n",
    "                \n",
    "                # Multi-stage matching logic\n",
    "                for pid, p_info in self.global_players.items():\n",
    "                    # Stage 1: High-confidence jersey match\n",
    "                    if p_info.get('confirmed_jersey') and current_features['jersey'] == p_info['confirmed_jersey']:\n",
    "                        best_id = pid; break\n",
    "                    \n",
    "                    # Stage 2: Appearance match\n",
    "                    color_sim = cv2.compareHist(current_features['color'], p_info['features']['color'], cv2.HISTCMP_CORREL)\n",
    "                    deep_sim = cosine_similarity(current_features['deep'].reshape(1, -1), p_info['features']['deep'].reshape(1, -1))[0][0]\n",
    "                    sim = 0.4 * color_sim + 0.6 * deep_sim\n",
    "                    if sim > best_score:\n",
    "                        best_score, best_id = sim, pid\n",
    "                \n",
    "                if best_id is None:\n",
    "                    best_id = self.next_permanent_id; self.next_permanent_id += 1\n",
    "                \n",
    "                if best_id in self.global_players:\n",
    "                    p_info = self.global_players[best_id]\n",
    "                    alpha = 0.1\n",
    "                    p_info['features']['color'] = (1-alpha) * p_info['features']['color'] + alpha * current_features['color']\n",
    "                    p_info['features']['deep'] = (1-alpha) * p_info['features']['deep'] + alpha * current_features['deep']\n",
    "                    p_info['ocr_buffer'] = p_info.get('ocr_buffer', []) + [current_features['jersey']]\n",
    "                    if len(p_info['ocr_buffer']) > 10: p_info['ocr_buffer'].pop(0)\n",
    "                else:\n",
    "                    self.global_players[best_id] = {'features': current_features, 'ocr_buffer': [current_features['jersey']]}\n",
    "                \n",
    "                # Validate OCR\n",
    "                ocr_counts = Counter([j for j in self.global_players[best_id]['ocr_buffer'] if j is not None])\n",
    "                if ocr_counts and ocr_counts.most_common(1)[0][1] >= self.ocr_validation_buffer:\n",
    "                    self.global_players[best_id]['confirmed_jersey'] = ocr_counts.most_common(1)[0][0]\n",
    "                \n",
    "                all_player_colors.append(current_features['dominant_color'])\n",
    "                frame_tracks[\"players\"].append({\"permanent_id\": best_id, \"bbox\": track['bbox'], \"jersey\": self.global_players[best_id].get('confirmed_jersey')})\n",
    "\n",
    "            long_tracks.append(frame_tracks)\n",
    "        \n",
    "        # Team classification\n",
    "        kmeans = KMeans(n_clusters=2, random_state=0, n_init=10).fit(all_player_colors)\n",
    "        team_colors = kmeans.cluster_centers_\n",
    "        for pid, p_info in self.global_players.items():\n",
    "            p_color = p_info['features']['dominant_color'].reshape(1, -1)\n",
    "            p_info['team'] = kmeans.predict(p_color)[0]\n",
    "\n",
    "        for frame_data in long_tracks:\n",
    "            for player in frame_data['players']:\n",
    "                player['team'] = int(self.global_players[player['permanent_id']]['team'])\n",
    "\n",
    "        cap.release()\n",
    "        with open(output_path, 'w') as f: json.dump(long_tracks, f, indent=2)\n",
    "        return long_tracks"
   ],
   "metadata": {
    "id": "reid_class"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "class AdvancedEventDetector:\n",
    "    def __init__(self, video_width, video_height, fps):\n",
    "        self.video_width = video_width\n",
    "        self.video_height = video_height\n",
    "        self.fps = fps if fps and fps > 0 else 30.0\n",
    "        self.player_history = {}\n",
    "        self.ball_history = []\n",
    "        self.goal_area = [video_width * 0.85, video_height * 0.2, video_width, video_height * 0.8]\n",
    "\n",
    "    def _update_history(self, players, ball, frame_id):\n",
    "        # Update Player History\n",
    "        for p in players:\n",
    "            pid = p['permanent_id']\n",
    "            center = np.array([(p['bbox'][0] + p['bbox'][2]) / 2, (p['bbox'][1] + p['bbox'][3]) / 2])\n",
    "            if pid not in self.player_history: self.player_history[pid] = []\n",
    "            self.player_history[pid].append({'frame_id': frame_id, 'center': center, 'team': p['team']})\n",
    "            if len(self.player_history[pid]) > self.fps * 2: self.player_history[pid].pop(0)\n",
    "        # Update Ball History\n",
    "        if ball: \n",
    "            center = np.array([(ball['bbox'][0] + ball['bbox'][2]) / 2, (ball['bbox'][1] + ball['bbox'][3]) / 2])\n",
    "            self.ball_history.append({'frame_id': frame_id, 'center': center})\n",
    "        if len(self.ball_history) > self.fps * 2: self.ball_history.pop(0)\n",
    "\n",
    "    def detect_events(self, long_tracks_path, detections_path, output_path):\n",
    "        with open(long_tracks_path, 'r') as f: all_tracks = json.load(f)\n",
    "        with open(detections_path, 'r') as f: all_detections = json.load(f)\n",
    "        events = []\n",
    "        \n",
    "        for i, frame_data in enumerate(tqdm(all_tracks, desc=\"Stage 4: Detecting Events\")):\n",
    "            frame_id = frame_data['frame_id']\n",
    "            players = frame_data['players']\n",
    "            ball = all_detections[i]['detections']['ball']\n",
    "            self._update_history(players, ball, frame_id)\n",
    "\n",
    "            if not ball or not self.ball_history or len(self.ball_history) < 2: continue\n",
    "\n",
    "            ball_pos = self.ball_history[-1]['center']\n",
    "            ball_vel = ball_pos - self.ball_history[-2]['center']\n",
    "            ball_speed = np.linalg.norm(ball_vel)\n",
    "\n",
    "            closest_player_dist = float('inf')\n",
    "            closest_player_id = None\n",
    "            for p in players:\n",
    "                dist = np.linalg.norm(self.player_history[p['permanent_id']][-1]['center'] - ball_pos)\n",
    "                if dist < closest_player_dist:\n",
    "                    closest_player_dist = dist\n",
    "                    closest_player_id = p['permanent_id']\n",
    "            \n",
    "            if closest_player_id and closest_player_dist < 50: # Player is close to the ball\n",
    "                # Shot detection\n",
    "                if ball_speed > 30 and self.goal_area[0] < ball_pos[0]:\n",
    "                    score = 1.0 * (ball_speed / 50)\n",
    "                    events.append(self._create_event(frame_id, 'shot', closest_player_id, score))\n",
    "                \n",
    "                # Pass detection\n",
    "                if 10 < ball_speed < 30:\n",
    "                    score = 0.5 * (ball_speed / 30)\n",
    "                    events.append(self._create_event(frame_id, 'pass', closest_player_id, score))\n",
    "\n",
    "        unique_events = self._deduplicate_events(events, self.fps * 3)\n",
    "        with open(output_path, 'w') as f: json.dump(unique_events, f, indent=2)\n",
    "        return unique_events\n",
    "\n",
    "    def _create_event(self, frame_id, event_type, player_id, score):\n",
    "        return {'frame_id': frame_id, 'timestamp': frame_id / self.fps, 'event_type': event_type, 'player_id': player_id, 'score': score}\n",
    "\n",
    "    def _deduplicate_events(self, events, window):\n",
    "        if not events: return []\n",
    "        sorted_events = sorted(events, key=lambda x: (-x['score'], x['frame_id']))\n",
    "        \n",
    "        final_events = []\n",
    "        processed_frames = set()\n",
    "        for event in sorted_events:\n",
    "            is_duplicate = False\n",
    "            for pf in processed_frames:\n",
    "                if abs(event['frame_id'] - pf) < window:\n",
    "                    is_duplicate = True; break\n",
    "            if not is_duplicate:\n",
    "                final_events.append(event)\n",
    "                processed_frames.add(event['frame_id'])\n",
    "        \n",
    "        return sorted(final_events, key=lambda x: x['frame_id'])"
   ],
   "metadata": {
    "id": "event_detector_class_v2"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "class VideoAssembler:\n",
    "    def __init__(self, clip_padding_seconds=2.0):\n",
    "        self.clip_padding = clip_padding_seconds\n",
    "        self.temp_dir = '/content/temp_clips'\n",
    "\n",
    "    def find_scenes(self, video_path):\n",
    "        video = open_video(video_path)\n",
    "        scene_manager = SceneManager()\n",
    "        scene_manager.add_detector(ContentDetector(threshold=25.0))\n",
    "        scene_manager.detect_scenes(video, show_progress=False)\n",
    "        return [(s[0].get_seconds(), s[1].get_seconds()) for s in scene_manager.get_scene_list()]\n",
    "\n",
    "    def assemble_highlight_reel(self, video_path, player_events_path, output_path, target_jersey_number, top_n):\n",
    "        with open(player_events_path, 'r') as f: all_events = json.load(f)\n",
    "        \n",
    "        # Find the permanent ID for the target jersey number\n",
    "        target_player_id = None\n",
    "        if 'reid' in globals() and reid.global_players:\n",
    "            for pid, p_info in reid.global_players.items():\n",
    "                if p_info.get('confirmed_jersey') == target_jersey_number:\n",
    "                    target_player_id = pid; break\n",
    "        \n",
    "        if not target_player_id: \n",
    "            print(f\"Could not find player with jersey number {target_jersey_number}\")\n",
    "            return False\n",
    "        \n",
    "        player_events = [e for e in all_events if e.get('player_id') == target_player_id]\n",
    "        if not player_events: return False\n",
    "\n",
    "        sorted_events = sorted(player_events, key=lambda x: -x['score'])\n",
    "        top_events = sorted_events[:top_n]\n",
    "\n",
    "        scenes = self.find_scenes(video_path)\n",
    "        clips_to_extract = []\n",
    "        for event in sorted(top_events, key=lambda x: x['timestamp']):\n",
    "            ts = event['timestamp']\n",
    "            for start, end in scenes:\n",
    "                if start <= ts <= end:\n",
    "                    if not any(abs(c['start'] - start) < 1.0 for c in clips_to_extract):\n",
    "                        clips_to_extract.append({'start': start, 'end': end, 'event': event})\n",
    "                    break\n",
    "        \n",
    "        shutil.rmtree(self.temp_dir, ignore_errors=True); os.makedirs(self.temp_dir, exist_ok=True)\n",
    "        clip_paths = []\n",
    "        for i, clip in enumerate(tqdm(clips_to_extract, desc=\"Stage 5: Assembling Reel\")):\n",
    "            clip_path = os.path.join(self.temp_dir, f\"clip_{i:04d}.mp4\")\n",
    "            # Add title overlay\n",
    "            text = f\"{clip['event']['event_type'].upper()} - Player {target_jersey_number}\"\n",
    "            command = ['ffmpeg', '-y', '-i', video_path, '-ss', str(clip['start']), '-to', str(clip['end']), \n",
    "                       '-vf', f\"drawtext=text='{text}':fontcolor=white:fontsize=48:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-text_w)/2:y=h-th-20\",\n",
    "                       '-c:a', 'copy', '-c:v', 'libx264', '-preset', 'fast', clip_path]\n",
    "            result = subprocess.run(command, capture_output=True, text=True)\n",
    "            if result.returncode == 0: clip_paths.append(clip_path)\n",
    "        \n",
    "        concat_list_path = os.path.join(self.temp_dir, \"concat_list.txt\")\n",
    "        with open(concat_list_path, 'w') as f: [f.write(f\"file '{os.path.basename(p)}'\\n\") for p in clip_paths]\n",
    "        \n",
    "        concat_command = ['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', concat_list_path, '-c', 'copy', output_path]\n",
    "        concat_result = subprocess.run(concat_command, cwd=self.temp_dir, capture_output=True, text=True)\n",
    "        \n",
    "        return concat_result.returncode == 0"
   ],
   "metadata": {
    "id": "assembler_class_v2"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "if 'video_path' in locals() and video_path:\n",
    "    detections_path = '/content/output/detections.json'\n",
    "    tracklets_path = '/content/output/tracklets.json'\n",
    "    long_tracks_path = '/content/output/long_player_track.json'\n",
    "    events_path = '/content/output/player_events.json'\n",
    "    highlight_path = f'/content/output/player_{TARGET_JERSEY_NUMBER}_highlights.mp4'\n",
    "\n",
    "    detector = SoccerObjectDetector(model_name=YOLO_MODEL_SIZE, conf_thresh=DETECTION_CONF_THRESHOLD, min_area=MIN_BBOX_AREA)\n",
    "    detections = detector.process_video(video_path, detections_path)\n",
    "    print(f\"✅ Detection complete! Processed {len(detections)} frames.\")\n",
    "\n",
    "    tracker = ByteTrack(high_thresh=TRACKING_HIGH_THRESH, low_thresh=TRACKING_LOW_THRESH, max_time_lost=TRACKING_MAX_TIME_LOST)\n",
    "    all_tracklets = []\n",
    "    for frame_data in tqdm(detections, desc=\"Stage 2: Tracking Players\"):\n",
    "        tracks = tracker.update(frame_data['detections']['players'])\n",
    "        all_tracklets.append({\"frame_id\": frame_data['frame_id'], \"tracks\": tracks})\n",
    "    with open(tracklets_path, 'w') as f: json.dump(all_tracklets, f, indent=2)\n",
    "    print(f\"✅ Tracking complete! Processed {len(all_tracklets)} frames.\")\n",
    "\n",
    "    reid = PlayerReID()\n",
    "    long_tracks = reid.process_tracklets(tracklets_path, video_path, long_tracks_path)\n",
    "    print(f\"✅ Re-ID & Team Classification complete! Identified {len(reid.global_players)} unique players.\")\n",
    "\n",
    "    cap = cv2.VideoCapture(video_path)\n",
    "    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))\n",
    "    video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))\n",
    "    fps = cap.get(cv2.CAP_PROP_FPS)\n",
    "    cap.release()\n",
    "\n",
    "    event_detector = AdvancedEventDetector(video_width, video_height, fps)\n",
    "    player_events = event_detector.detect_events(long_tracks_path, detections_path, events_path)\n",
    "    print(f\"✅ Advanced event detection complete! Found {len(player_events)} unique highlight events.\")\n",
    "\n",
    "    assembler = VideoAssembler()\n",
    "    success = assembler.assemble_highlight_reel(video_path, events_path, highlight_path, TARGET_JERSEY_NUMBER, TOP_N_EVENTS)\n",
    "    if success:\n",
    "        print(f\"✅ Highlight reel saved to: {highlight_path}\")\n",
    "    else:\n",
    "        print(\"❌ Failed to create highlight reel\")\n",
    "else:\n",
    "    print(\"❌ No video path available. Please run the upload cell first.\")"
   ],
   "metadata": {
    "id": "run_pipeline"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "source": [
    "## 📥 5. Download Results"
   ],
   "metadata": {
    "id": "download_markdown"
   }
  },
  {
   "cell_type": "code",
   "source": [
    "from google.colab import files\n",
    "\n",
    "if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):\n",
    "    files.download(highlight_path)\n",
    "    print(f\"✅ Downloaded: {os.path.basename(highlight_path)}\")\n",
    "else:\n",
    "    print(f\"❌ Could not download highlight reel. File not found or creation failed.\")\n",
    "\n",
    "if os.path.exists('/content/output/long_player_track.json'):\n",
    "    files.download('/content/output/long_player_track.json')\n",
    "    print(\"✅ Downloaded: long_player_track.json\")\n",
    "\n",
    "if os.path.exists('/content/output/player_events.json'):\n",
    "    files.download('/content/output/player_events.json')\n",
    "    print(\"✅ Downloaded: player_events.json\")\n",
    "\n",
    "print(\"\\n🎉 Pipeline complete! Check your browser's downloads folder.\")"
   ],
   "metadata": {
    "id": "download_results"
   },
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "code",
   "source": [
    "print(\"📊 PIPELINE SUMMARY\")\n",
    "print(\"=\" * 50)\n",
    "\n",
    "if 'video_path' in locals() and video_path and os.path.exists('/content/output/long_player_track.json') and os.path.exists('/content/output/player_events.json'):\n",
    "    with open('/content/output/long_player_track.json', 'r') as f:\n",
    "        long_tracks_summary = json.load(f)\n",
    "    with open('/content/output/player_events.json', 'r') as f:\n",
    "        player_events_summary = json.load(f)\n",
    "\n",
    "    print(f\"📹 Video processed: {os.path.basename(video_path)}\")\n",
    "    print(f\"🎯 Target player jersey: {TARGET_JERSEY_NUMBER}\")\n",
    "    print(f\"🖼️ Total frames processed: {len(long_tracks_summary)}\")\n",
    "    print(f\"✨ Total unique events detected: {len(player_events_summary)}\")\n",
    "\n",
    "    if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):\n",
    "        print(f\"⏱️ Highlight reel generated successfully for Top {TOP_N_EVENTS} events.\")\n",
    "        file_size = os.path.getsize(highlight_path) / (1024 * 1024)\n",
    "        print(f\"📁 Output file size: {file_size:.1f} MB\")\n",
    "    else:\n",
    "        print(\"⏱️ Highlight reel was not generated.\")\n",
    "else:\n",
    "    print(\"Could not generate summary. One or more output files are missing.\")\n",
    "\n",
    "print(\"=\" * 50)\n",
    "print(\"🎉 Pipeline finished!\")"
   ],
   "metadata": {
    "id": "summary"
   },
   "execution_count": null,
   "outputs": []
  }
 ]
}


In [ ]:
from google.colab import files
import os

# Download highlight reel
if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
    files.download(highlight_path)
    print(f"✅ Downloaded: {os.path.basename(highlight_path)}")
else:
    print(f"❌ Could not download highlight reel. File not found or creation failed.")

# Download all JSON outputs
json_files = [
    '/content/output/detections.json',
    '/content/output/tracklets.json', 
    '/content/output/long_player_track.json',
    '/content/output/player_events.json'
]

for json_file in json_files:
    if os.path.exists(json_file):
        files.download(json_file)
        print(f"✅ Downloaded: {os.path.basename(json_file)}")
    else:
        print(f"❌ Not found: {os.path.basename(json_file)}")

print("\n📥 Download complete! Check your browser's downloads folder.")


In [ ]:
from google.colab import files
import os, json, cv2
from tqdm.notebook import tqdm

os.makedirs('/content/output', exist_ok=True)

detections_path = '/content/output/detections.json'
tracklets_path = '/content/output/tracklets.json'
long_tracks_path = '/content/output/long_player_track.json'
events_path = '/content/output/player_events.json'
highlight_path = f'/content/output/player_{TARGET_JERSEY_NUMBER}_highlights.mp4'

outputs_required = [detections_path, tracklets_path, long_tracks_path, events_path]

ran = False
if 'video_path' in globals() and video_path:
    if not all(os.path.exists(p) for p in outputs_required):
        detector = SoccerObjectDetector(model_name=YOLO_MODEL_SIZE, conf_thresh=DETECTION_CONF_THRESHOLD, min_area=MIN_BBOX_AREA)
        detections = detector.process_video(video_path, detections_path)
        tracker = ByteTrack(high_thresh=TRACKING_HIGH_THRESH, low_thresh=TRACKING_LOW_THRESH, max_time_lost=TRACKING_MAX_TIME_LOST)
        all_tracklets = []
        for frame_data in tqdm(detections, desc='Stage 2: Tracking Players'):
            tracks = tracker.update(frame_data['detections']['players'])
            all_tracklets.append({'frame_id': frame_data['frame_id'], 'tracks': tracks})
        with open(tracklets_path, 'w') as f: json.dump(all_tracklets, f, indent=2)
        reid = PlayerReID()
        reid.process_tracklets(tracklets_path, video_path, long_tracks_path)
        cap = cv2.VideoCapture(video_path)
        vw = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); vh = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)); fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        ed = AdvancedEventDetector(vw, vh, fps)
        ed.detect_events(long_tracks_path, detections_path, events_path)
        assembler = VideoAssembler()
        success = assembler.assemble_highlight_reel(video_path, events_path, highlight_path, TARGET_JERSEY_NUMBER, TOP_N_EVENTS)
        ran = True
else:
    print('❌ No video uploaded. Please run the upload cell first.')

if 'success' in locals() and success and os.path.exists(highlight_path):
    files.download(highlight_path)
    print(f"✅ Downloaded: {os.path.basename(highlight_path)}")

for json_file in outputs_required:
    if os.path.exists(json_file):
        files.download(json_file)
        print(f"✅ Downloaded: {os.path.basename(json_file)}")

print("\n📥 Done.")


In [ ]:
print("📊 DETAILED PIPELINE SUMMARY")
print("=" * 60)

if 'video_path' in locals() and video_path and os.path.exists('/content/output/long_player_track.json') and os.path.exists('/content/output/player_events.json'):
    with open('/content/output/long_player_track.json', 'r') as f:
        long_tracks_summary = json.load(f)
    with open('/content/output/player_events.json', 'r') as f:
        player_events_summary = json.load(f)

    print(f"📹 Video processed: {os.path.basename(video_path)}")
    print(f"🎯 Target player jersey: {TARGET_JERSEY_NUMBER}")
    print(f"🖼️ Total frames processed: {len(long_tracks_summary)}")
    print(f"✨ Total unique events detected: {len(player_events_summary)}")

    # Player statistics
    if 'reid' in globals() and reid.global_players:
        print(f"👥 Unique players identified: {len(reid.global_players)}")
        confirmed_jerseys = [p.get('confirmed_jersey') for p in reid.global_players.values() if p.get('confirmed_jersey')]
        print(f"🔢 Players with confirmed jersey numbers: {len(confirmed_jerseys)}")
        if confirmed_jerseys:
            print(f"🔢 Jersey numbers found: {sorted(confirmed_jerseys)}")

    # Event breakdown
    event_types = {}
    for event in player_events_summary:
        event_type = event.get('event_type', 'unknown')
        event_types[event_type] = event_types.get(event_type, 0) + 1
    
    print(f"\n📌 Events by type:")
    for event_type, count in event_types.items():
        print(f"   • {event_type}: {count}")

    # Target player events
    target_player_id = None
    if 'reid' in globals() and reid.global_players:
        for pid, p_info in reid.global_players.items():
            if p_info.get('confirmed_jersey') == TARGET_JERSEY_NUMBER:
                target_player_id = pid; break

    target_events = [e for e in player_events_summary if target_player_id and e.get('player_id') == target_player_id]
    print(f"\n🎯 Events for player {TARGET_JERSEY_NUMBER}: {len(target_events)}")
    
    if target_events:
        target_event_types = {}
        for event in target_events:
            event_type = event.get('event_type', 'unknown')
            target_event_types[event_type] = target_event_types.get(event_type, 0) + 1
        for event_type, count in target_event_types.items():
            print(f"   • {event_type}: {count}")

    # Highlight reel status
    if 'success' in locals() and success and 'highlight_path' in locals() and os.path.exists(highlight_path):
        print(f"\n⏱️ Highlight reel generated successfully!")
        print(f"📁 Output file: {os.path.basename(highlight_path)}")
        file_size = os.path.getsize(highlight_path) / (1024 * 1024)
        print(f"📊 File size: {file_size:.1f} MB")
        print(f"🏆 Top {TOP_N_EVENTS} events included")
    else:
        print(f"\n⏱️ Highlight reel was not generated.")
        if target_events:
            print(f"💡 Tip: If the player was detected, try increasing TOP_N_EVENTS or lowering thresholds.")
else:
    print("❌ Could not generate summary. One or more output files are missing.")

print("=" * 60)
print("🎉 Pipeline finished!")
